<a href="https://colab.research.google.com/github/Yuki-fujiif/pythonwebscraping/blob/main/20230607WEBscraping0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# 使用ライブラリ
import difflib
import requests 
from bs4 import BeautifulSoup
import os
from datetime import datetime
import pytz


# スプレッドシートを開く
spreadsheet = gc.open("test sheet01") # スプレットシートのファイル名を入力
worksheet1 = spreadsheet.get_worksheet(0)  # Sheet1

col = 2  # B列

# ワークシート1の列Bを順番に処理
for row in range(2, worksheet1.row_count + 1):

    url = worksheet1.cell(row, col).value  # スプレットシートのURLを取得

    if not url:  # URLがなければループを抜ける
        break

    # 前回のシートと現在のシートを取得または作成
    last_sheet_title = f"LastSheet{row}"
    curr_sheet_title = f"CurrSheet{row}"

    last_sheet = None
    curr_sheet = None
    for sheet in spreadsheet.worksheets():
        if sheet.title == last_sheet_title:
            last_sheet = sheet
        elif sheet.title == curr_sheet_title:
            curr_sheet = sheet

    if last_sheet is None:
        last_sheet = spreadsheet.add_worksheet(title=last_sheet_title, rows="100", cols="20")
    if curr_sheet is None:
        curr_sheet = spreadsheet.add_worksheet(title=curr_sheet_title, rows="100", cols="20")

    # 現在の日付を取得
    current_date = datetime.now(pytz.timezone('Asia/Tokyo')).strftime('%Y-%m-%d')

    #ページのHMTLを取得。できなかった場合、エラー処理をする。
    try:
        response = requests.get(url)
    except requests.exceptions.ConnectionError:
            worksheet1.update_cell(row, col + 1, f"URLが見つかりませんでした。")
            continue

    soup = BeautifulSoup(response.content, 'html.parser', from_encoding='utf-8')  # 取得したHTMLを解析

    try:
        curr_sheet.update_cell(1,1, (soup.get_text()))  # 差分比較用に現在のシートに書き込む
    except gspread.exceptions.APIError as e:  # 書き込みエラーを捕捉
        worksheet1.update_cell(row, col + 1, f"エラー: {e}")
        continue

    # 前回のシートと現在のシートを比較
    last_sheet_values = last_sheet.get_all_values()
    curr_sheet_values = curr_sheet.get_all_values()

    diff = difflib.unified_diff(['\t'.join(row) for row in last_sheet_values], ['\t'.join(row) for row in curr_sheet_values]) 
    diff_list = list(diff)

    if len(diff_list) > 0:
        worksheet1.update_cell(row, col + 1, f"{current_date} 更新があります。") # 変更があった場合、変更日付付きの通知をマークする

    # 現在のシートの内容を前回のシートにコピー
    last_sheet.clear() 
    for i, row in enumerate(curr_sheet_values):
        last_sheet.insert_row(row, i+1)


#------------------------------------------------------------------

In [ ]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

#使用ライブラリ
import difflib
import requests 
from bs4 import BeautifulSoup
import os
from datetime import datetime

# 現在の日付を取得します
current_date = datetime.now().strftime('%Y-%m-%d')

##スプレッドシートを開き、巡回先URLリストの先頭のURLにアクセスする①

spreadsheet = gc.open("test sheet01") #スプレットシートのファイル名を入力

worksheet1 = spreadsheet.get_worksheet(0)  # Sheet1
worksheet2 = spreadsheet.get_worksheet(1)  # Sheet2前回更新時データ
worksheet3 = spreadsheet.get_worksheet(2)  # Sheet3現在時刻データ

row = 3
col = 2  # B列

url = worksheet1.cell(row, col).value #スプレットシートB.3のURLを取得
response = requests.get(url) #ページのHTMLを取得する②
soup = BeautifulSoup(response.content, 'html.parser', from_encoding='utf-8') #取得したHTMLを解析する③


worksheet3.update_cell(1,1, (soup.get_text())) #差分比較用にシート3に書き込む。

sheet2_values = worksheet2.get_all_values()
sheet3_values = worksheet3.get_all_values()

diff = difflib.unified_diff(['\t'.join(row) for row in sheet2_values], ['\t'.join(row) for row in sheet3_values]) #シート2 シート3の差分を検出
diff_list = list(diff)

if len(diff_list) > 0:
  worksheet1.update_cell(row, col + 1, f"{current_date} 更新があります。") #変更があった場合、変更日付付きの通知をマークする

  worksheet2.clear() #シート2の内容を消去する
  for i, row in enumerate(sheet3_values):
        worksheet2.insert_row(row, i+1) #シート3の全ての行をシート2にコピー


In [ ]:
##使用ライブラリメモ
#import requests 
#from bs4 import BeautifulSoup
#import os
#from datetime import datetime
#import difflib
#import pytz